Human Activity Tracker

Human Activity Detection is the problem of predicting what a person is doing based on a trace of their movement using sensors. The "Human Activity Detection #Dataset" includes data collected from 34 subjects, each of whom were asked to perform 18 tasks for 3 minutes each. Each subject had a smartwatch placed on his/her dominant hand and a smartphone in their pocket. The data collection was controlled by a custom-made app that ran on the #smartphone and #smartwatch. The sensor data that was collected was from the accelerometer and gyroscope on both the smartphone and smartwatch, yielding four total sensors. The sensor data was collected at a rate of 20 Hz (i.e., every 50ms). The smartphone was either the Google Nexus 5/5X or Samsung Galaxy S5 running Android 6.0 (Marshmallow). The smartwatch was the LG G Watch running Android Wear 1.5.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('wisdm-dataset/raw/phone/accel'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


wisdm-dataset/raw/phone/accel\.DS_Store
wisdm-dataset/raw/phone/accel\data_1600_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1601_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1602_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1603_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1604_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1605_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1606_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1607_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1608_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1609_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1610_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1611_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1612_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1613_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1614_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1615_accel_phone.txt
wisdm-dataset/raw/phone/accel\data_1616_accel_phone.txt
wisdm-da

In [2]:
from platform import python_version

print(python_version())


3.9.13


In [3]:
from __future__ import print_function
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv1D, Conv2D, LSTM, MaxPooling2D, TimeDistributed, ConvLSTM2D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import np_utils


Phone Accelerometer Train/test

In [4]:
columns=['user','activity','time','x','y','z']

data_phone_accel_sum = pd.DataFrame(data=None,columns=columns)
for dirname, _, filenames in os.walk('wisdm-dataset/raw/phone/accel'):
    for filename in filenames:
        if filename == (".DS_Store"):
            continue
        df = pd.read_csv('wisdm-dataset/raw/phone/accel/'+filename , sep=",", header=None)
        temp=pd.DataFrame(data=df.values, columns=columns)
        data_phone_accel_sum=pd.concat([data_phone_accel_sum,temp])

In [5]:
data_phone_accel_sum['z'] = data_phone_accel_sum['z'].str.replace(';','')
data_phone_accel_sum['activity'].value_counts()
data_phone_accel_sum['x']=data_phone_accel_sum['x'].astype('float')
data_phone_accel_sum['y']=data_phone_accel_sum['y'].astype('float')
data_phone_accel_sum['z']=data_phone_accel_sum['z'].astype('float')
data_phone_accel_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4804403 entries, 0 to 160778
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 256.6+ MB


Phone Gyro files import Train/test

In [6]:

data_phone_gyro_sum = pd.DataFrame(data=None,columns=columns)
for dirname, _, filenames in os.walk('wisdm-dataset/raw/phone/gyro'):
    for filename in filenames:
        if filename == (".DS_Store"):
            continue
        df = pd.read_csv('wisdm-dataset/raw/phone/gyro/'+filename , sep=",", header=None)
        temp=pd.DataFrame(data=df.values, columns=columns)
        data_phone_gyro_sum=pd.concat([data_phone_gyro_sum,temp])

In [7]:
data_phone_gyro_sum['z'] = data_phone_gyro_sum['z'].str.replace(';','')

data_phone_gyro_sum['x']=data_phone_gyro_sum['x'].astype('float')
data_phone_gyro_sum['y']=data_phone_gyro_sum['y'].astype('float')
data_phone_gyro_sum['z']=data_phone_gyro_sum['z'].astype('float')

data_phone_gyro_sum['activity'].value_counts()
data_phone_gyro_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3608635 entries, 0 to 80347
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 192.7+ MB


Watch Gyro files import train/test

In [8]:

data_watch_gyro_sum = pd.DataFrame(data=None,columns=columns)
for dirname, _, filenames in os.walk('wisdm-dataset/raw/watch/gyro'):
    for filename in filenames:
        if filename == (".DS_Store"):
            continue
        df = pd.read_csv('wisdm-dataset/raw/watch/gyro/'+filename , sep=",", header=None)
        temp=pd.DataFrame(data=df.values, columns=columns)
        data_watch_gyro_sum=pd.concat([data_watch_gyro_sum,temp])

In [9]:
data_watch_gyro_sum['z'] = data_watch_gyro_sum['z'].str.replace(';','')
data_watch_gyro_sum['x']=data_watch_gyro_sum['x'].astype('float')
data_watch_gyro_sum['y']=data_watch_gyro_sum['y'].astype('float')
data_watch_gyro_sum['z']=data_watch_gyro_sum['z'].astype('float')

data_watch_gyro_sum['activity'].value_counts()
data_watch_gyro_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440342 entries, 0 to 66555
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 183.7+ MB


Watch accelorometer files import train test

In [10]:

data_watch_accel_sum = pd.DataFrame(data=None,columns=columns)
for dirname, _, filenames in os.walk('wisdm-dataset/raw/watch/accel'):
    for filename in filenames:
        if filename == (".DS_Store"):
            continue
        df = pd.read_csv('wisdm-dataset/raw/watch/accel/'+filename , sep=",", header=None)
        temp=pd.DataFrame(data=df.values, columns=columns)
        data_watch_accel_sum=pd.concat([data_watch_accel_sum,temp])

In [11]:
data_watch_accel_sum['z'] = data_watch_accel_sum['z'].str.replace(';','')
data_watch_accel_sum['x']=data_watch_accel_sum['x'].astype('float')
data_watch_accel_sum['y']=data_watch_accel_sum['y'].astype('float')
data_watch_accel_sum['z']=data_watch_accel_sum['z'].astype('float')

data_watch_accel_sum['activity'].value_counts()
data_watch_accel_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3777046 entries, 0 to 66591
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 201.7+ MB


Combining Phone accel and gyro data

In [12]:
df_phone = pd.DataFrame(data=None, columns=columns)
df_phone['user']= data_phone_accel_sum['user'].head(3608635)
df_phone['activity']= data_phone_accel_sum['activity'].head(3608635)
df_phone['time']= data_phone_accel_sum['time'].head(3608635)
df_phone['x'] = data_phone_gyro_sum['x'].values + data_phone_accel_sum['x'].head(3608635).values
df_phone['y'] = data_phone_gyro_sum['y'].values + data_phone_accel_sum['y'].head(3608635).values
df_phone['z'] = data_phone_gyro_sum['z'].values + data_phone_accel_sum['z'].head(3608635).values

Combining watch acccel and gyro data

In [13]:
df_watch = pd.DataFrame(data=None, columns=columns)
df_watch['user']= data_watch_accel_sum['user'].head(3440342)
df_watch['activity']= data_watch_accel_sum['activity'].head(3440342)
df_watch['time']= data_watch_accel_sum['time'].head(3440342)
df_watch['x'] = data_watch_gyro_sum['x'].values + data_watch_accel_sum['x'].head(3440342).values
df_watch['y'] = data_watch_gyro_sum['x'].values + data_watch_accel_sum['y'].head(3440342).values
df_watch['z'] = data_watch_gyro_sum['x'].values + data_watch_accel_sum['z'].head(3440342).values

In [14]:
df_phone['activity'].value_counts()

A    217194
K    213640
H    207882
I    207678
M    207215
G    206994
E    206992
B    205784
D    201973
P    201173
O    200160
R    196265
L    194054
S    193668
C    193031
Q    188703
F    183740
J    182489
Name: activity, dtype: int64

In [15]:
df_watch['activity'].value_counts()

E    198506
K    197861
D    195012
P    194799
Q    194782
O    192099
I    192041
H    191460
M    191170
A    190717
G    190713
S    189727
C    189302
F    187129
R    187126
B    187115
L    185677
J    185106
Name: activity, dtype: int64

Combining Phone and Watch Data

In [16]:
df_phone_watch = pd.DataFrame(data=None, columns=columns)
df_phone_watch['user']= df_phone['user'].head(3440342)
df_phone_watch['activity']= df_phone['activity'].head(3440342)
df_phone_watch['time']= df_phone['time'].head(3440342)
df_phone_watch['x'] = df_watch['x'].values + df_phone['x'].head(3440342).values
df_phone_watch['y'] = df_watch['y'].values + df_phone['y'].head(3440342).values
df_phone_watch['z'] = df_watch['z'].values + df_phone['z'].head(3440342).values

In [17]:
df_phone_watch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440342 entries, 0 to 59277
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 183.7+ MB


In [18]:
df_phone_watch['activity'].value_counts()

A    208259
K    204705
M    198279
E    198059
B    196849
G    194809
D    193039
C    193031
P    192236
O    191229
H    190017
I    189812
R    187332
L    185123
S    184736
F    183740
Q    179770
J    169317
Name: activity, dtype: int64

In [19]:
Fs = 20

In [20]:
activities = df_phone_watch['activity'].value_counts().index

In [21]:
df_phone_watch = df_phone_watch.drop('time', axis=1)

In [22]:
df_phone_watch['activity'].value_counts()

A    208259
K    204705
M    198279
E    198059
B    196849
G    194809
D    193039
C    193031
P    192236
O    191229
H    190017
I    189812
R    187332
L    185123
S    184736
F    183740
Q    179770
J    169317
Name: activity, dtype: int64

In [23]:
df_a = df_phone_watch[df_phone_watch['activity']=='A'].head(174604)
df_m = df_phone_watch[df_phone_watch['activity']=='M'].head(174604)
df_k = df_phone_watch[df_phone_watch['activity']=='K'].head(174604)
df_p = df_phone_watch[df_phone_watch['activity']=='P'].head(174604)
df_e = df_phone_watch[df_phone_watch['activity']=='E'].head(174604)
df_o = df_phone_watch[df_phone_watch['activity']=='O'].head(174604)
df_c = df_phone_watch[df_phone_watch['activity']=='C'].head(174604)
df_d = df_phone_watch[df_phone_watch['activity']=='D'].head(174604)
df_l = df_phone_watch[df_phone_watch['activity']=='L'].head(174604)
df_b = df_phone_watch[df_phone_watch['activity']=='B'].head(174604)
df_h = df_phone_watch[df_phone_watch['activity']=='H'].head(174604)
df_f = df_phone_watch[df_phone_watch['activity']=='F'].head(174604)
df_g = df_phone_watch[df_phone_watch['activity']=='G'].head(174604)
df_q = df_phone_watch[df_phone_watch['activity']=='Q'].head(174604)
df_r = df_phone_watch[df_phone_watch['activity']=='R'].head(174604)
df_s = df_phone_watch[df_phone_watch['activity']=='S'].head(174604)
df_i = df_phone_watch[df_phone_watch['activity']=='I'].head(174604)
df_j = df_phone_watch[df_phone_watch['activity']=='J']

In [24]:
balanced_data = pd.DataFrame()
balanced_data = pd.concat([df_a,df_m,df_k,df_p,df_e,df_o,df_c,df_d,df_l,df_b,df_h,df_f,df_g,df_q,df_r,df_s,df_i,df_j]) 

In [25]:
balanced_data['activity'].value_counts()

A    174604
M    174604
I    174604
S    174604
R    174604
Q    174604
G    174604
F    174604
H    174604
B    174604
L    174604
D    174604
C    174604
O    174604
E    174604
P    174604
K    174604
J    169317
Name: activity, dtype: int64

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
label = LabelEncoder()
balanced_data['label'] = label.fit_transform(balanced_data['user'])
balanced_data['activity'] = label.fit_transform(balanced_data['activity']) 
balanced_data = balanced_data.drop('user', axis=1)
# balanced_data['activity'] = pd.get_dummies(balanced_data['activity'])
balanced_data

,activity,x,y,z,label
0,0,6.188597,8.814006,10.455713,0
1,0,3.605272,10.013322,8.263343,0
2,0,4.606045,11.376722,8.513901,0
3,0,2.718174,13.761097,5.978487,0
4,0,1.299151,14.153046,3.168536,0
...,...,...,...,...,...
36003,9,-13.463638,-8.818773,-2.320777,40
36004,9,-13.582652,-9.013937,-2.345182,40
36005,9,-13.629185,-8.722762,-2.201741,40
36006,9,-13.675635,-9.399319,-2.620970,40


In [28]:
label.classes_

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'O', 'P', 'Q', 'R', 'S'], dtype=object)

**Statndardize Data**

In [29]:
from sklearn.preprocessing import StandardScaler

x = balanced_data[['x','y','z']]
y = balanced_data['label']
scaler = StandardScaler()
x = scaler.fit_transform(x)

scaled_x = pd.DataFrame(data=x, columns=['x','y','z'])
scaled_x['label'] = y.values

scaled_x

,x,y,z,label
0,0.644592,1.511277,1.015689,0
1,0.362824,1.640989,0.721891,0
2,0.471980,1.788448,0.755468,0
3,0.266066,2.046331,0.415699,0
4,0.111291,2.088722,0.039139,0
...,...,...,...,...
3137580,-1.498913,-0.395803,-0.696481,40
3137581,-1.511894,-0.416911,-0.699752,40
3137582,-1.516970,-0.385419,-0.680529,40
3137583,-1.522036,-0.458592,-0.736710,40


**Frame Preparation**

In [30]:
import scipy.stats as stats

In [31]:
Fs=20
frame_size = Fs*4 #80
hop_size = Fs*2 #40

In [32]:
def get_frames(df, frame_size, hop_size):
    
    N_FEATURES = 4
    frames = []
    labels = []
    for i in range(0,len(df )- frame_size, hop_size):
        x = df['x'].values[i: i+frame_size]
        y = df['y'].values[i: i+frame_size]
        z = df['z'].values[i: i+frame_size]
        activ = balanced_data['activity'].values[i: i+frame_size]
        
        label = stats.mode(df['label'][i: i+frame_size])[0][0]
        frames.append([activ,x,y,z])
        labels.append(label)
        
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)
    
    return frames, labels

In [33]:
x,y = get_frames(scaled_x, frame_size, hop_size)

C:\Users\Admin\AppData\Local\Temp\ipykernel_51624\16734148.py:12: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  label = stats.mode(df['label'][i: i+frame_size])[0][0]


In [34]:
 x.shape, y.shape

((78438, 80, 4), (78438,))

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.30, random_state = 0, stratify = y)

In [37]:
y_train.shape

(54906,)

In [38]:
x_train.shape, x_test.shape

((54906, 80, 4), (23532, 80, 4))

In [39]:
x_train[0].shape, x_test[0].shape

((80, 4), (80, 4))

In [40]:
x_train = x_train.reshape(54906, 80, 4,1)
x_test = x_test.reshape(23532, 80, 4,1)
x.shape

(78438, 80, 4)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

 
#Flatten into 2D array
x_train_2d = x_train.reshape(x_train.shape[0], -1)
x_test_2d = x_test.reshape(x_test.shape[0], -1)


# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Create a base model
base_model = RandomForestClassifier(random_state=0)

# Instantiate the grid search
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(x_train_2d, y_train)

# Print the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)





In [48]:
from sklearn.ensemble import RandomForestClassifier
#Flatten into 2D array
x_train_2d = x_train.reshape(x_train.shape[0], -1)
x_test_2d = x_test.reshape(x_test.shape[0], -1)

# Create a base model
base_model = RandomForestClassifier(random_state=0, criterion = 'entropy', max_depth = 30, max_features = 'log2', n_estimators = 200)

# Fit the grid search to the data
base_model.fit(x_train_2d, y_train)


RandomForestClassifier(criterion='entropy', max_depth=30, max_features='log2',
                       n_estimators=200, random_state=0)

In [49]:
import tikzplotlib

def plot_accuracy(model):
    plt.figure(figsize=(10, 5))
    plt.plot(model.history['accuracy'], 'g--', label='Accuracy of training data') 
    plt.plot(model.history['val_accuracy'], 'g', label='Accuracy of validation data')
    #plt.title('Model Accuracy and Loss', fontsize=17)
    plt.legend(loc='center left', shadow=True, fontsize=17)
    plt.ylabel('Training Progress', fontsize=17)
    plt.xlabel('Training Epoch', fontsize=17)
    plt.ylim(0)
    plt.legend()
    plt.show()

In [50]:
from keras.models import load_model
best_model = base_model

In [52]:
from sklearn.preprocessing import LabelBinarizer

label_as_binary = LabelBinarizer()
y_train_labels = label_as_binary.fit_transform(y_train)
x_train_1 = x_train.reshape(x_train.shape[0], 1, x_train.shape[1], x_train.shape[2])

In [53]:
# reshape data into time steps of sub-sequences
y_test_label = label_as_binary.fit_transform(y_test)
x_test_1 = x_test_2d
y_pred_test = best_model.predict(x_test_1)


In [ ]:
# Take the class with the highest probability from the test predictions
y_pred_test = best_model.predict(x_test_1)
max_y_pred_test = np.argmax(y_pred_test, axis = 0)
max_y_test = np.argmax(y_test_label, axis = 0)
#Confusion Matrix
from pycm import *
cm = ConfusionMatrix(actual_vector= y_test_label, predict_vector= y_pred_test)
cm.plot(cmap=plt.cm.Blues,normalized=True,number_label=True,plot_lib="seaborn")

<AxesSubplot:title={'center':'Confusion Matrix (Normalized)'}, xlabel='Predicted Classes', ylabel='Actual Classes'>

In [ ]:
# Take the class with the highest probability from the test predictions
y_pred_test = best_model.predict(x_test_1)
max_y_pred_test = np.argmax(y_pred_test)
max_y_test = np.argmax(y_test)

print(metrics.classification_report(y_test, y_pred_test))
